In [1]:
import json
import pandas as pd
import numpy as np
from dfply import *
import altair.vega.v5 as alt


In [2]:
# Prepare the GraphQL client
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)

client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

In [3]:
# Generate the GraphQL query to retrieve up to 100 outputs of University of Oxford, with at least 100 views each.
query_params = {
    "rorId" : "https://ror.org/052gg0110",
    "maxOutputs": 100,
    "minViews" : 100
}

query = gql("""query getOutputs($rorId: ID!, $maxOutputs: Int!, $minViews: Int!)
{
 organization(id: $rorId) {
    id
    name
    alternateName
    citationCount
    viewCount
    downloadCount
    works(hasViews: $minViews, first: $maxOutputs) {
      totalCount
      published {
        title
        count
      }
      resourceTypes {
        title
        count
      }
      nodes {
        id
        type
        publisher
        publicationYear
        titles {
          title
        }
        citations {
           nodes {
             id
             titles {
                title
             }
           }
        }
        creators {
          id
          name
          affiliation {
            id
            name
          }
        }
        citationCount
        viewCount
        downloadCount
      }
    }
  }
}
""")

In [4]:
# pd = nb_setup.setup_pandas(escape_latex=False)


In [5]:
# data = client.execute(query, variable_values=json.dumps(query_params))

In [6]:
def get_data():
    response_string = '{"data":{"work":{"relatedIdentifiers":[{"relatedIdentifier":"10.5281\/zenodo.2648713","relatedIdentifierType":"DOI","relationType":"Cites"},{"relatedIdentifier":"10.5281\/zenodo.2648740","relatedIdentifierType":"DOI","relationType":"Cites"},{"relatedIdentifier":"10.5281\/zenodo.2648800","relatedIdentifierType":"DOI","relationType":"IsSupplementTo"},{"relatedIdentifier":"10.5281\/zenodo.2648751","relatedIdentifierType":"DOI","relationType":"HasVersion"}],"creators":[{"id":"https:\/\/orcid.org\/0000-0002-4537-6648","name":"Leidinger, Moritz","affiliation":[{"id":null}]}],"publisher":"Zenodo","identifiers":[{"identifier":"https:\/\/zenodo.org\/record\/2648751","identifierType":"URL"}],"fundingReferences":null,"repository":{"id":"cern.zenodo","re3dataId":"10.17616\/R3QP53","repositoryType":["other"]},"citations":{"totalCount":0,"published":[],"nodes":[]},"references":{"published":[],"nodes":[{ "id": "https://doi.org/10.15468/dl.ojaesd", "formattedCitation": "Occdownload Gbif.Org. (2015). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.OJAESD</a>", "repository": { "name": "Global Biodiversity Information Facility", "re3dataId": "10.17616/R3J014", "id": "gbif.gbif" }, "registrationAgency": { "name": null, "id": null }, "member": { "name": "Global Biodiversity Information Facility", "id": "gbif" } }, { "id": "https://doi.org/10.15468/dl.uq8ndo", "formattedCitation": "Occdownload Gbif.Org. (2018). <i>GBIF Occurrence Download</i> [Data set]. The Global Biodiversity Information Facility. https://doi.org/10.15468/DL.UQ8NDO</a>", "repository": { "name": "Global Biodiversity Information Facility", "re3dataId": "10.17616/R3J014", "id": "gbif.gbif" }, "registrationAgency": { "name": "DataCite", "id": "datacite" }, "member": { "name": "Global Biodiversity Information Facility", "id": "gbif" } }]}}}}'
    return json.loads(response_string)["data"]["work"]

In [9]:
x = pd.DataFrame(get_data()["references"]["nodes"],columns=get_data()["references"]["nodes"][1].keys())

In [10]:
x

id  \
0  https://doi.org/10.15468/dl.ojaesd   
1  https://doi.org/10.15468/dl.uq8ndo   

                                   formattedCitation  \
0  Occdownload Gbif.Org. (2015). <i>GBIF Occurren...   
1  Occdownload Gbif.Org. (2018). <i>GBIF Occurren...   

                                          repository  \
0  {'name': 'Global Biodiversity Information Faci...   
1  {'name': 'Global Biodiversity Information Faci...   

                       registrationAgency  \
0              {'name': None, 'id': None}   
1  {'name': 'DataCite', 'id': 'datacite'}   

                                              member  
0  {'name': 'Global Biodiversity Information Faci...  
1  {'name': 'Global Biodiversity Information Faci...